In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from  xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
all_dataset = pd.read_csv("dataset.CSV")

In [3]:
all_dataset

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Target,ID
0,41,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,11,other,-,840801042
1,36,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,26,other,-,840803046
2,32,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,36,other,S,840803047
3,60,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,26,other,-,840803048
4,77,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,21,other,-,840803068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9106,43,M,f,f,f,f,f,f,f,f,...,130,t,0.84,t,155,f,?,other,B,861030019
9107,60,M,t,f,f,f,f,f,f,f,...,155,t,0.78,t,199,f,?,other,N,861030031
9108,42,F,f,f,f,f,f,f,f,f,...,178,t,0.71,t,251,f,?,other,A,861114085
9109,63,M,t,f,f,f,f,f,f,f,...,96,t,0.92,t,104,f,?,other,-,861127039


In [4]:
all_dataset

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Target,ID
0,41,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,11,other,-,840801042
1,36,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,26,other,-,840803046
2,32,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,36,other,S,840803047
3,60,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,26,other,-,840803048
4,77,F,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,21,other,-,840803068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9106,43,M,f,f,f,f,f,f,f,f,...,130,t,0.84,t,155,f,?,other,B,861030019
9107,60,M,t,f,f,f,f,f,f,f,...,155,t,0.78,t,199,f,?,other,N,861030031
9108,42,F,f,f,f,f,f,f,f,f,...,178,t,0.71,t,251,f,?,other,A,861114085
9109,63,M,t,f,f,f,f,f,f,f,...,96,t,0.92,t,104,f,?,other,-,861127039


In [5]:
def handleDuplicated(df):
    if df["ID"].duplicated().sum() == 0 :
        print("There aren't duplicates")
    elif (df["ID"].duplicated().sum()) < len(df) / 100:
        df["ID"].drop_duplicates(keep="first", inplace=True)
        print("duplicates were less than the 1% of all the data, they have been dropped")
    else:
        index_duplicated = df["ID"].duplicated().index
        print("duplicates are more than the 1% of all the data, they have been preserved")
        print(index_duplicated)

In [6]:
handleDuplicated(all_dataset)

There aren't duplicates


In [7]:
all_dataset['sex'] = all_dataset['sex'].map({'F': 1, 'M': 0})

In [8]:
all_dataset.replace(['A','B','C','D'],"hyperthyroid",inplace = True)
all_dataset.replace(['E','F','G','H'],"hypothyroid",inplace = True)

In [9]:
for value in set(all_dataset['Target']):
    if(value != 'hypothyroid' and value != 'hyperthyroid'):
        all_dataset.replace(value,'negative',inplace=True)

In [10]:
all_dataset

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Target,ID
0,41,1.0,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,11,other,negative,840801042
1,36,1.0,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,26,other,negative,840803046
2,32,1.0,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,36,other,negative,840803047
3,60,1.0,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,26,other,negative,840803048
4,77,1.0,f,f,f,f,f,f,f,f,...,?,f,?,f,?,t,21,other,negative,840803068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9106,43,0.0,f,f,f,f,f,f,f,f,...,130,t,0.84,t,155,f,?,other,hyperthyroid,861030019
9107,60,0.0,t,f,f,f,f,f,f,f,...,155,t,0.78,t,199,f,?,other,negative,861030031
9108,42,1.0,f,f,f,f,f,f,f,f,...,178,t,0.71,t,251,f,?,other,hyperthyroid,861114085
9109,63,0.0,t,f,f,f,f,f,f,f,...,96,t,0.92,t,104,f,?,other,negative,861127039


In [13]:
all_dataset = all_dataset.replace({"t":1,"f":0, "y":1, "n":0, "hypothyroid":1, "negative":0,"hyperthyroid":2, "F":1, "M":0})
display(all_dataset.dtypes)

age                            int64
sex                          float64
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment                 int64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                  int64
psych                          int64
TSH_measured                   int64
TSH                           object
T3_measured                    int64
T3                            object
TT4_measured                   int64
TT4                           object
T4U_measured                   int64
T4U                           object
FTI_measured                   int64
FTI                           object
TBG_measured                   int64
T

In [14]:
all_dataset=all_dataset.replace({"?":np.NAN})

In [15]:
all_dataset.isna().sum()

age                             0
sex                           305
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
sick                            0
pregnant                        0
thyroid_surgery                 0
I131_treatment                  0
query_hypothyroid               0
query_hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH_measured                    0
TSH                           842
T3_measured                     0
T3                           2588
TT4_measured                    0
TT4                           440
T4U_measured                    0
T4U                           806
FTI_measured                    0
FTI                           799
TBG_measured                    0
TBG                          8762
referral_source                 0
Target        

In [16]:
all_dataset

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_medication,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,Target,ID
0,41,1.0,0,0,0,0,0,0,0,0,...,NaN,0,NaN,0,NaN,1,11,other,0,840801042
1,36,1.0,0,0,0,0,0,0,0,0,...,NaN,0,NaN,0,NaN,1,26,other,0,840803046
2,32,1.0,0,0,0,0,0,0,0,0,...,NaN,0,NaN,0,NaN,1,36,other,0,840803047
3,60,1.0,0,0,0,0,0,0,0,0,...,NaN,0,NaN,0,NaN,1,26,other,0,840803048
4,77,1.0,0,0,0,0,0,0,0,0,...,NaN,0,NaN,0,NaN,1,21,other,0,840803068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9106,43,0.0,0,0,0,0,0,0,0,0,...,130,1,0.84,1,155,0,NaN,other,2,861030019
9107,60,0.0,1,0,0,0,0,0,0,0,...,155,1,0.78,1,199,0,NaN,other,0,861030031
9108,42,1.0,0,0,0,0,0,0,0,0,...,178,1,0.71,1,251,0,NaN,other,2,861114085
9109,63,0.0,1,0,0,0,0,0,0,0,...,96,1,0.92,1,104,0,NaN,other,0,861127039


In [17]:
cols = all_dataset.columns[all_dataset.dtypes.eq('object')]
all_dataset[cols] = all_dataset[cols].apply(pd.to_numeric, errors='coerce')
display(all_dataset.dtypes)

age                            int64
sex                          float64
on_thyroxine                   int64
query_on_thyroxine             int64
on_antithyroid_medication      int64
sick                           int64
pregnant                       int64
thyroid_surgery                int64
I131_treatment                 int64
query_hypothyroid              int64
query_hyperthyroid             int64
lithium                        int64
goitre                         int64
tumor                          int64
hypopituitary                  int64
psych                          int64
TSH_measured                   int64
TSH                          float64
T3_measured                    int64
T3                           float64
TT4_measured                   int64
TT4                          float64
T4U_measured                   int64
T4U                          float64
FTI_measured                   int64
FTI                          float64
TBG_measured                   int64
T

In [18]:
all_dataset['TSH']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
9106    0.005
9107    0.005
9108    0.005
9109    0.005
9110    0.005
Name: TSH, Length: 9111, dtype: float64

In [19]:
del all_dataset['TBG']
del all_dataset['referral_source']
del all_dataset['TBG_measured']

In [20]:
all_dataset.isna().sum()

age                             0
sex                           305
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
sick                            0
pregnant                        0
thyroid_surgery                 0
I131_treatment                  0
query_hypothyroid               0
query_hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH_measured                    0
TSH                           842
T3_measured                     0
T3                           2588
TT4_measured                    0
TT4                           440
T4U_measured                    0
T4U                           806
FTI_measured                    0
FTI                           799
Target                          0
ID                              0
dtype: int64

In [21]:
all_dataset.dropna(axis = 0, thresh = 20, inplace = True)
all_dataset.isna().sum()

age                             0
sex                           305
on_thyroxine                    0
query_on_thyroxine              0
on_antithyroid_medication       0
sick                            0
pregnant                        0
thyroid_surgery                 0
I131_treatment                  0
query_hypothyroid               0
query_hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH_measured                    0
TSH                           842
T3_measured                     0
T3                           2588
TT4_measured                    0
TT4                           440
T4U_measured                    0
T4U                           806
FTI_measured                    0
FTI                           799
Target                          0
ID                              0
dtype: int64

In [22]:
corr_values = abs(all_dataset[all_dataset.columns[0:]].corr()['Target'][:])
corr_values = corr_values.drop('Target')
corr_values = corr_values[corr_values > 0.04]
display(corr_values)
display(corr_values.index)

sex                   0.049134
on_thyroxine          0.086364
query_hyperthyroid    0.146984
psych                 0.048629
TSH                   0.361807
T3                    0.290967
TT4_measured          0.051982
TT4                   0.104277
FTI                   0.158703
Name: Target, dtype: float64

Index(['sex', 'on_thyroxine', 'query_hyperthyroid', 'psych', 'TSH', 'T3',
       'TT4_measured', 'TT4', 'FTI'],
      dtype='object')

In [23]:
def holdout(dataframe):
  x = dataframe[corr_values.index]
  y = dataframe['Target']
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42) 
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = holdout(all_dataset)

In [24]:
classifiers = {
    "XGBClassifier" : XGBClassifier(learning_rate=0.01),
    "CatBoostClassifier" : CatBoostClassifier(max_depth=4,verbose=0),
}

In [25]:
def classification(classifiers, X_train, X_test, y_train, y_test):
    # Creo un dataframe per visualizzare i risultati calcolati
  res = pd.DataFrame(columns=["Classifier", 
                                "Accuracy", 
                                "Precision", 
                                "Recall", 
                                "FScore"])
  for name, clf in classifiers.items():
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            pr, rc, fs, sup = metrics.precision_recall_fscore_support(y_test, y_pred, average='macro')
            res = res.append({"Classifier": name,"Accuracy": round(metrics.accuracy_score(y_test, y_pred), 4),
                              "Precision": round(pr, 4), "Recall":round(rc, 4), "FScore":round(fs, 4)}, ignore_index=True)
            print("Confusion matrix for: ", name)
            display(confusion_matrix(y_test, y_pred))
  res.set_index("FScore", inplace=True)
  res.sort_values(by="FScore", ascending=False, inplace=True)   
  return res

display(classification(classifiers, X_train, X_test, y_train, y_test))

C:\Users\Asus\Anaconda3\envs\py3.7-TF2.0\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:22:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Confusion matrix for:  XGBClassifier


array([[2488,   15,    8],
       [   2,  155,    0],
       [  20,    0,   46]], dtype=int64)

Confusion matrix for:  CatBoostClassifier


array([[2481,   18,   12],
       [   2,  155,    0],
       [  16,    0,   50]], dtype=int64)

,Classifier,Accuracy,Precision,Recall
FScore,,,,
0.9037,CatBoostClassifier,0.9824,0.8984,0.9110
0.9019,XGBClassifier,0.9835,0.9183,0.8917


In [26]:
display(all_dataset.shape)
all_dataset.Target.value_counts()

(9111, 28)

0    8393
1     536
2     182
Name: Target, dtype: int64

In [27]:
data_alt = all_dataset.interpolate(method = 'spline', order = 3,limit_direction='both')
display(data_alt.isna().sum())

age                          0
sex                          0
on_thyroxine                 0
query_on_thyroxine           0
on_antithyroid_medication    0
sick                         0
pregnant                     0
thyroid_surgery              0
I131_treatment               0
query_hypothyroid            0
query_hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
TSH_measured                 0
TSH                          0
T3_measured                  0
T3                           0
TT4_measured                 0
TT4                          0
T4U_measured                 0
T4U                          0
FTI_measured                 0
FTI                          0
Target                       0
ID                           0
dtype: int64

In [28]:
classifiers1 = {
    "XGBClassifier" : XGBClassifier(learning_rate=0.01),
    "CatBoostClassifier" : CatBoostClassifier(max_depth=4,verbose=0),
    "Nearest Neighbors" : KNeighborsClassifier(4),
    "Decision Tree" : DecisionTreeClassifier(class_weight = 'balanced'),
    "Random Forest": RandomForestClassifier(class_weight = 'balanced',random_state = 1),
    "ExtraTrees": ExtraTreesClassifier(class_weight = 'balanced',random_state = 1),
    "MLPClassifier": MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation="relu",random_state=1)
}

In [29]:
corr_values = abs(data_alt[data_alt.columns[0:]].corr()['Target'][:])
corr_values = corr_values.drop('Target')
corr_values = corr_values[corr_values > 0.04]
display(corr_values)

sex                   0.049651
on_thyroxine          0.086364
query_hyperthyroid    0.146984
psych                 0.048629
T3                    0.255713
TT4_measured          0.051982
FTI                   0.095481
Name: Target, dtype: float64

In [30]:

X_train_alt, X_test_alt, y_train_alt, y_test_alt = holdout(data_alt)

display(classification(classifiers1,X_train_alt, X_test_alt, y_train_alt, y_test_alt))

C:\Users\Asus\Anaconda3\envs\py3.7-TF2.0\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:22:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Confusion matrix for:  XGBClassifier


array([[2492,   12,    7],
       [ 135,   22,    0],
       [  32,    0,   34]], dtype=int64)

Confusion matrix for:  CatBoostClassifier


array([[2480,   21,   10],
       [ 120,   37,    0],
       [  32,    0,   34]], dtype=int64)

...

Confusion matrix for:  ExtraTrees


array([[2383,  110,   18],
       [ 102,   55,    0],
       [  33,    0,   33]], dtype=int64)

Confusion matrix for:  MLPClassifier


array([[2489,   20,    2],
       [ 127,   30,    0],
       [  50,    0,   16]], dtype=int64)

,Classifier,Accuracy,Precision,Recall
FScore,,,,
0.6423,CatBoostClassifier,0.9331,0.7843,0.5795
0.6294,Random Forest,0.9133,0.7022,0.5854
0.6178,ExtraTrees,0.9038,0.6423,0.5998
0.6175,Decision Tree,0.9012,0.6422,0.6008
0.6100,XGBClassifier,0.9320,0.8045,0.5492
0.5441,MLPClassifier,0.9272,0.8075,0.4749
0.4796,Nearest Neighbors,0.9206,0.6983,0.4310
